In [23]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Загрузка данных
df = pd.read_csv('2023-11-07_Opros_pervokursnikov_3.csv')  

# Используйте столбцы с первым и вторым вопросами как признаки
features_question_1 = df.iloc[:, 3]
features_question_2 = df.iloc[:, 4]

# Преобразование категориальных признаков в числовой формат
label_encoder = LabelEncoder()
features_encoded_question_1 = label_encoder.fit_transform(features_question_1)

# Создание нового DataFrame с закодированными данными
df_encoded = pd.DataFrame({
    'Ответ_вопроса_1': features_encoded_question_1,
    'Ответ_вопроса_2': features_question_2
})

# Присвоение весов ответам на первый вопрос
weights_question_1 = {
    0: 1,  # "Я активно ищу возможности для участия"
    1: 3,  # "Мне интересно, но я ещё не участвовал"
    2: 2,  # "Мне это не интересно"
    3: 4   # "Я не знаю, что такое научные конференции."
}

# Присвоение весов ответам на второй вопрос
weights_question_2 = {
    'Технологии и инновации': 1,
    'Гуманитарные науки': 3,
    'Естественные науки': 3,
    'Социальные науки': 3,
    'Любая тема интересует': 2
}

# Применение весов к ответам на оба вопроса
df_encoded['Важность_вопроса_1'] = df_encoded['Ответ_вопроса_1'].map(weights_question_1)
df_encoded['Важность_вопроса_2'] = df_encoded['Ответ_вопроса_2'].map(weights_question_2)

# Дополнительная логика для формирования кластеров
additional_conditions = (
    (df['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?'] == "Мне интересно, но я ещё не участвовал") &
    (df['В какой тематике вы бы хотели участвовать в научной конференции?'] == 'Технологии и инновации')
)

# Установка важности для ответов, соответствующих дополнительным условиям
df_encoded.loc[additional_conditions, 'Важность_вопроса_1'] = 1  # Меняем важность вопроса_1 для соответствующих условий

# Кластеризация с использованием Agglomerative Clustering
n_clusters = 4  # Задаем количество кластеров
agg_cluster = AgglomerativeClustering(n_clusters=n_clusters)
labels_agg = agg_cluster.fit_predict(df_encoded[['Ответ_вопроса_1', 'Важность_вопроса_1', 'Важность_вопроса_2']])

# Добавление столбца 'Cluster' с метками кластеров в исходный датасет
df['Cluster'] = labels_agg

# Вывод результатов
df.to_csv('result1_df.csv', index=False)


In [27]:
test1_question = df[['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?','В какой тематике вы бы хотели участвовать в научной конференции?', 'Cluster']]
test1_question.to_csv('test1_question.csv', index=False)

cluster_counts = test1_question['Cluster'].value_counts()
print(cluster_counts)

Cluster
0    59
1    49
3    31
2    11
Name: count, dtype: int64


In [28]:
cluster_distribution = test1_question.groupby(['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?','В какой тематике вы бы хотели участвовать в научной конференции?', 'Cluster']).size().reset_index(name='Counts')

total_responses = cluster_distribution['Counts'].sum()
cluster_distribution['Percentage'] = (cluster_distribution['Counts'] / total_responses) * 100

cluster_distribution.to_csv('test1_distribution.csv', index=False)
